# Classification Using A CNN (and CIFAR-10)
---

In this notebook we implement a convolutional neural network to classify low quality images (from the CIFAR-10) dataset where each entry belongs to exactly one of 10 classes.

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import datetime

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

device:  cuda:0


In [2]:
# Get training and validation datasets from PyTorch
# Pass both to dataloaders for sampling batches. 

data_path = 'C:\\Users\\Kyle\\Documents\\GitHub\\data\\'
# Define a transform to normalize the 3 channels of each image to 0.5 and 0.5 mean, std dev.
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(data_path, train=True, download=True,  transform=transform)
val_set = torchvision.datasets.CIFAR10(data_path, train=False, download=True,  transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# === Helper Functions, members ===

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Show image 
def imshow(img):
    # display an image
    img = img / 2 + 0.5 # unnormalize image to [0,1]
    npimg = img.numpy()
    # Display image by reordering channels to match pyplot's expectation
    plt.imshow(np.transpose(npimg, (1,2,0)))
    
# Training loop
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
    # loop over epochs
    for epoch in range(1, n_epochs+1):
        loss_train = 0.0
        
        # iter over training batch
        for imgs, labels in train_loader:
            # convert tensors to device
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            y_pred = model(imgs)
            
            loss = loss_fn(y_pred, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
                
        # display training loss
        if epoch % 10 == 0 or epoch == 1: 
            loss = loss_train / len(train_loader)
            print(f'{datetime.datetime.now()} Epoch: {epoch}, Training loss: {loss:.3f}')
            validate(model, train_loader, val_loader)
            
# Validation loop
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("validate", val_loader)]:
        correct_pred, total = 0, 0
        # disable tracking of operations 
        with torch.no_grad():
            for imgs, labels in loader:
                # convert tensors to device
                imgs = imgs.to(device)
                labels = labels.to(device)
                
                y_preds = model(imgs)
                _, predicted = torch.max(y_preds, dim=1)
                total += labels.shape[0]
                # compare agreements between predicted and labels, sum and add to running total. 
                correct_pred += int((predicted==labels).sum())
        print(f'Accuracy: {name} {correct_pred/total:.3f}')

### Define CNN

In [4]:
###
# Define CNN
###

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        # take 3x32x32 --> 6x32x32 
        self.conv1 = nn.Conv2d(3, 6, kernel_size=3, padding=1)
        # downsample 6x32x32 --> 6x16x16
        self.pool1 = nn.MaxPool2d(2, 2)
        # convolve 6x16x16 --> 8x16x16
        self.conv2 = nn.Conv2d(6, 8, kernel_size=3, padding=1)
        # convolve 8x16x16 --> 8x8x8
        self.pool2 = nn.MaxPool2d(2, 2)
        # flatten 8x8x8 --> 64x1
        self.fc1 = nn.Linear(8*8*8, 64)
        # flatten 64x1 --> 32x1
        self.fc2 = nn.Linear(64, 32)
        # flatten 64x1 --> 32x1
        self.fc3 = nn.Linear(32, 10)
        # flatten 32x1 --> 10x1
        
    def forward(self, x):
        # Apply convolution, activation function, and pooling to input
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        # reshape x to match first linear layer
        x = x.view(-1, 8*8*8)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [5]:
model = CNN()
model.to(device)

opt = torch.optim.SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

training_loop(100, opt, model, loss_fn, train_loader,val_loader)

2020-10-16 16:22:03.420076 Epoch: 1, Training loss: 2.309
Accuracy: train 0.100
Accuracy: validate 0.100
2020-10-16 16:24:33.217186 Epoch: 10, Training loss: 2.302
Accuracy: train 0.101
Accuracy: validate 0.100
2020-10-16 16:27:08.486036 Epoch: 20, Training loss: 2.287
Accuracy: train 0.166
Accuracy: validate 0.168
2020-10-16 16:29:44.068601 Epoch: 30, Training loss: 2.136
Accuracy: train 0.235
Accuracy: validate 0.243
2020-10-16 16:32:20.049918 Epoch: 40, Training loss: 1.929
Accuracy: train 0.307
Accuracy: validate 0.310
2020-10-16 16:34:57.293646 Epoch: 50, Training loss: 1.805
Accuracy: train 0.352
Accuracy: validate 0.358
2020-10-16 16:37:37.194800 Epoch: 60, Training loss: 1.671
Accuracy: train 0.406
Accuracy: validate 0.416
2020-10-16 16:40:18.914544 Epoch: 70, Training loss: 1.581
Accuracy: train 0.431
Accuracy: validate 0.436
2020-10-16 16:43:01.575840 Epoch: 80, Training loss: 1.511
Accuracy: train 0.459
Accuracy: validate 0.457
2020-10-16 16:45:37.208600 Epoch: 90, Training 

In [6]:
training_loop(100, opt, model, loss_fn, train_loader,val_loader)

2020-10-16 16:48:54.663112 Epoch: 1, Training loss: 1.378
Accuracy: train 0.507
Accuracy: validate 0.497
2020-10-16 16:51:29.790516 Epoch: 10, Training loss: 1.329
Accuracy: train 0.511
Accuracy: validate 0.505
2020-10-16 16:54:09.050409 Epoch: 20, Training loss: 1.283
Accuracy: train 0.543
Accuracy: validate 0.529
2020-10-16 16:56:47.296551 Epoch: 30, Training loss: 1.244
Accuracy: train 0.548
Accuracy: validate 0.538
2020-10-16 16:59:25.730105 Epoch: 40, Training loss: 1.210
Accuracy: train 0.564
Accuracy: validate 0.552
2020-10-16 17:02:04.282921 Epoch: 50, Training loss: 1.179
Accuracy: train 0.576
Accuracy: validate 0.553
2020-10-16 17:04:42.777710 Epoch: 60, Training loss: 1.150
Accuracy: train 0.582
Accuracy: validate 0.562
2020-10-16 17:07:21.694377 Epoch: 70, Training loss: 1.121
Accuracy: train 0.603
Accuracy: validate 0.571
2020-10-16 17:10:00.005311 Epoch: 80, Training loss: 1.097
Accuracy: train 0.611
Accuracy: validate 0.581
2020-10-16 17:12:37.872334 Epoch: 90, Training 

In [7]:
training_loop(100, opt, model, loss_fn, train_loader,val_loader)

2020-10-16 17:31:43.049340 Epoch: 1, Training loss: 1.050
Accuracy: train 0.606
Accuracy: validate 0.566
2020-10-16 17:34:18.936782 Epoch: 10, Training loss: 1.032
Accuracy: train 0.627
Accuracy: validate 0.587
2020-10-16 17:37:06.843294 Epoch: 20, Training loss: 1.012
Accuracy: train 0.642
Accuracy: validate 0.604
2020-10-16 17:39:52.425294 Epoch: 30, Training loss: 0.995
Accuracy: train 0.633
Accuracy: validate 0.593
2020-10-16 17:42:32.071045 Epoch: 40, Training loss: 0.977
Accuracy: train 0.653
Accuracy: validate 0.608
2020-10-16 17:45:12.748039 Epoch: 50, Training loss: 0.961
Accuracy: train 0.655
Accuracy: validate 0.608
2020-10-16 17:47:52.584042 Epoch: 60, Training loss: 0.945
Accuracy: train 0.657
Accuracy: validate 0.606
2020-10-16 17:50:46.135725 Epoch: 70, Training loss: 0.929
Accuracy: train 0.666
Accuracy: validate 0.613
2020-10-16 17:53:27.032729 Epoch: 80, Training loss: 0.916
Accuracy: train 0.673
Accuracy: validate 0.613
2020-10-16 17:56:07.572725 Epoch: 90, Training 

In [8]:
training_loop(100, opt, model, loss_fn, train_loader,val_loader)

2020-10-16 18:06:07.116517 Epoch: 1, Training loss: 0.888
Accuracy: train 0.677
Accuracy: validate 0.612
2020-10-16 18:08:34.174513 Epoch: 10, Training loss: 0.876
Accuracy: train 0.690
Accuracy: validate 0.618
2020-10-16 18:11:14.892565 Epoch: 20, Training loss: 0.865
Accuracy: train 0.697
Accuracy: validate 0.622
2020-10-16 18:13:56.332562 Epoch: 30, Training loss: 0.853
Accuracy: train 0.702
Accuracy: validate 0.621
2020-10-16 18:16:37.491341 Epoch: 40, Training loss: 0.842
Accuracy: train 0.689
Accuracy: validate 0.614
2020-10-16 18:19:18.518339 Epoch: 50, Training loss: 0.831
Accuracy: train 0.703
Accuracy: validate 0.621
2020-10-16 18:22:01.353352 Epoch: 60, Training loss: 0.820
Accuracy: train 0.713
Accuracy: validate 0.622
2020-10-16 18:24:50.903193 Epoch: 70, Training loss: 0.810
Accuracy: train 0.708
Accuracy: validate 0.617
2020-10-16 18:27:40.784209 Epoch: 80, Training loss: 0.801
Accuracy: train 0.707
Accuracy: validate 0.617
2020-10-16 18:30:31.503314 Epoch: 90, Training 

In [9]:
# save model 
data_path = 'C:\\Users\\Kyle\\Documents\\GitHub\\Learning-Repo\\PyTorch\\models\\'
torch.save(model.state_dict(), data_path+'cifar_10_test.pt')